In [ ]:
# Imports

import pandas as pd
import itertools
import random
import numpy as np
import difflib
from functools import total_ordering
from collections import Counter
from itertools import groupby



def random_product(*args, repeat=1):
    "Random selection from itertools.product(*args, **kwds)"
    pools = [tuple(pool) for pool in args] * repeat
    return tuple(map(np.random.choice, pools))

In [ ]:
class Participant:

    def __init__(self, name, phone, nationality):
        self.name = name
        self.phone = phone
        self.nationality = nationality


    def __str__(self) -> str:
        return f"({self.name}, {self.nationality})"

class Pair:

    def __init__(self, id, participant_1, participant_2, coloc, dish):
        self.id = id
        self.p1 = participant_1
        self.p2 = participant_2
        self.coloc = coloc
        self.dish = dish

    def get_nationalities(self):
        return [self.p1.nationality, self.p2.nationality]


    def __str__(self) -> str:
        return f"{self.id}: {self.p1} and {self.p2} cooking {self.dish} at {self.coloc}"

@total_ordering
class Group:
    def __init__(self, cooker, eating_pairs) -> None:

        self.cooker = cooker
        self.eating_pairs = eating_pairs

        self.nat_count = self.get_nat_count()
        self.similarity = self.calc_similarity()

    def get_nat_count(self):
        nats = self.cooker.get_nationalities() + self.eating_pairs[0].get_nationalities() + self.eating_pairs[1].get_nationalities()
        return Counter(nats)

    def calc_similarity(self):
        return sum([(self.nat_count[nat]-1)**2 for nat in self.nat_count])/25

    def __gt__(self, other):
        return self.similarity < other.similarity
    
    def __eq__(self, other):
        return self.similarity == other.similarity

    def __str__(self) -> str:
        return f"{self.cooker}\n{self.eating_pairs[0]}\n{self.eating_pairs[1]}"


In [ ]:
def generate_eating_combinations(eating_pairs_set):

    eating_pairs_combinations = set()

    for i, pair_1 in enumerate(eating_pairs_set):
        for j, pair_2 in enumerate(eating_pairs_set):
            if i != j:
                eating_pairs_combinations.add(frozenset({pair_1, pair_2}))

    return eating_pairs_combinations


def generate_event(cooking_set, eating_set):

    eating_combination_set = generate_eating_combinations(eating_set)

    possible_events = []
    for cooking_pair in cooking_set:
        for eating_comb in eating_combination_set:
            possible_events.append(Group(cooking_pair,tuple(eating_comb)))
            
    return possible_events

In [ ]:
# Setup and data loading

data = pd.read_csv("data.csv").drop(columns=["Date"])

appetizers_df = data.loc[data["Dish"].str.contains("Appetizer")]
main_df = data.loc[data["Dish"].str.contains("Main dish")]
dessert_df = data.loc[data["Dish"].str.contains("Dessert")]


appetizers_set = set()
main_set = set()
desserts_set = set()

for i, pair in data.iterrows():
    p1 = Participant(pair["P1 Name"],pair["P1 Phone"], pair["P1 Nationality"])
    p2 = Participant(pair["P2 Name"],pair["P2 Phone"], pair["P2 Nationality"])

    pair = Pair(i,p1,p2,pair["Coloc"],pair["Dish"])


    if pair.dish == "Appetizer":
        appetizers_set.add(pair)
    elif pair.dish == "Main dish":
        main_set.add(pair)
    elif pair.dish == "Dessert":
        desserts_set.add(pair)



In [ ]:
event1 = generate_event(appetizers_set, set.union(main_set,desserts_set))
event1.sort()
event1.reverse()

result = list(map(Group.calc_similarity,event1))
best_interval = [len(list(group)) for value, group in groupby(result)][0]
best = event1[0:best_interval]

print(np.random.choice(best))

for p in best:
    print(p.similarity,"\n",p,"\n")


In [ ]:


for p in event1:
    print(p.similarity,"\n",p,"\n")

In [ ]:
person = Participant("Jorge","9999","Spain")
person2 = Participant("Jorge","9999","Brazil")
person3 = Participant("Jorge","9999","Bali")
person4 = Participant("Jorge","9999","Normal")
person5 = Participant("Jorge","9999","Super")
person6 = Participant("Jorge","9999","Lala")

pair = Pair(1,person,person2,"465","appetizer")
pair2 = Pair(1,person3,person4,"465","main")
pair3 = Pair(1,person5,person6,"465","dessert")
pair4 = Pair(1,person5,person6,"465","dessert")

group = Group(pair,(pair2,pair3))
group = Group(pair,(pair,pair))

appetizers_set = {pair}
main_set = {pair2}
dessert_set = {pair3,pair4}

eating_comb = generate_eating_combinations(set.union(main_set,dessert_set))
event1 = generate_event(appetizers_set, set.union(main_set,dessert_set))

event1.sort()
print(event1)

